THIS NOTEBOOK WAS RUN ON COLAB WITH THE T4 GPU TO MAKE THE EXPERIMENTS RUN FASTER

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import copy
import torch
from transformers import BertTokenizerFast, BertModel, BertForMaskedLM
from torch.nn import functional as F
import torch.nn as nn
import pickle
from tqdm import tqdm
import numpy as np
import math

# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')
model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
#CREATE A FILENAME FOR THE REPORT
report = open("fixed-random-probes-p3.txt", "w")

with open('/content/drive/MyDrive/bertstuff/john_sentences.txt', 'r') as file:
    # Read lines from the file and store them in a list
    input_sentences = file.read().splitlines()  #116

# obtaining distance matrix
with open('/content/drive/MyDrive/bertstuff/distance_john', 'rb') as f:
    distance_matrices = pickle.load(f)

distance_dict = {}  #58
for i in range(len(input_sentences)):
    if input_sentences[i] not in distance_dict:
        distance_dict[input_sentences[i]] = []
    distance_dict[input_sentences[i]].append(distance_matrices[i])

def apply_transformation(matrix, vector):
    transformed_vector = torch.mm(torch.transpose(matrix, 0, 1), vector)
    return transformed_vector


valid_words = []
invalid_words = []
sentences = list(input_sentences)  #58
sentences = [sentences[i] for i in range(0, len(sentences), 2)]

import json
with open("/content/drive/MyDrive/bertstuff/dataset.json") as f:
    data = json.load(f)

for sent in data:
    attachment1_words = sent["attachment1_words"]
    attachment2_words = sent["attachment2_words"]

    min_length = min(len(attachment1_words), len(attachment2_words))
    attachment1_words = attachment1_words[:min_length]
    attachment2_words = attachment2_words[:min_length]

    valid_words.append(attachment1_words)
    invalid_words.append(attachment2_words)


def b_loss(hidden_vectors, dist_context, theta, original_hidden_vectors, A_mask):
    transformed_hidden = apply_transformation(b_matrix, torch.transpose(hidden_vectors, 0, 1))  # Permute for GPU compatibility
    transformed_hidden = torch.transpose(transformed_hidden, 0, 1)  # Permute back
    hidden_matrix = transformed_hidden.unsqueeze(1).expand(transformed_hidden.size()[0],
                                                            transformed_hidden.size()[0],
                                                            transformed_hidden.size()[1])

    hidden_square_scaled = (torch.linalg.norm(dist_context)/torch.linalg.norm(hidden_matrix))*hidden_matrix

    diffs = torch.linalg.norm(torch.transpose(hidden_square_scaled, 0, 1) - hidden_square_scaled, ord=2, dim=2)**2
    diffs.requires_grad_(True)
    loss2 = torch.mean(torch.tensor(A_mask, dtype=torch.float64).to(hidden_vectors.device) * (diffs - dist_context)**2)
    loss1 = torch.norm((hidden_vectors - original_hidden_vectors)**2, 2)

    loss = theta * loss1 + (1 - theta) * loss2
    return loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE " + str(device))


for layer in range(11,12):
  report.write(f'LAYER is {layer} \n')

  submodel = BertForMaskedLM.from_pretrained('bert-base-cased', output_hidden_states=True, return_dict = True, num_hidden_layers = 11)
  submodel.to(device)

  for rank in [32, 64]:
    for theta in [0.3, 0.6, 1]:
      report.write(f'\nNEW RUN: layer {layer} rank {rank} theta {theta}\n')
      print(f'\nNEW RUN: layer {layer} rank {rank} theta {theta}\n')
      increase_score = 0
      decrease_score = 0

      for sent_index in range(len(sentences)):
        sent = sentences[sent_index]
        print("index " + str(sent_index) + " " + "this sent")
        input = tokenizer(sent, return_tensors="pt").to(device)
        orig_output = model(**input)
        mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
        orig_logits = orig_output.logits
        orig_softmax = F.softmax(orig_logits, dim = -1)
        orig_mask_word = orig_softmax[0, mask_index, :]


        #made a dictionary of all the original BERT model words and predictions
        orig_predictions = {}

        orig_values_second_dimension = orig_mask_word[0, :]
        orig_num_predictions = orig_mask_word[0, :].shape[0]

        orig_all_words = torch.topk(orig_mask_word, orig_num_predictions, dim = 1)[1][0]
        orig_all_probabilites = torch.topk(orig_mask_word, orig_num_predictions, dim = 1)[0][0]
        orig_counter = 0
        for token in orig_all_words:
            word = tokenizer.decode([token])
            orig_predictions[word] = orig_all_probabilites[orig_counter].item()
            orig_counter += 1

        for i in range(2):
            report.write(sent + "\n")
            input = tokenizer(sent, return_tensors="pt").to(device)
            output = submodel(**input)
            mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)

            A_mask = np.not_equal(distance_dict[sent][0], distance_dict[sent][1]).astype(int)
            A_mask = torch.tensor(A_mask, device=device)

            b_matrix_path = f"/content/drive/MyDrive/bertstuff/probes/r{str(rank)}/predictor_random.params"
            import_b = torch.load(b_matrix_path, map_location=device)
            b_matrix = import_b['proj'].to(device)

            hidden_states = output.hidden_states[-1].to(device)
            optimized_hidden_vectors = hidden_states[0][1:-1].clone()
            og_hidden_vectors = optimized_hidden_vectors.clone()
            optimized_hidden_vectors = optimized_hidden_vectors.requires_grad_(True)
            optimized_hidden_vectors.retain_grad()

            distance_first_context = torch.tensor(distance_dict[sent][i], dtype=torch.float32, device=device)
            distance_first_context = distance_first_context**2
            distance_first_context.requires_grad_(True)

            initial_loss = b_loss(optimized_hidden_vectors, distance_first_context, theta, og_hidden_vectors, A_mask)

            # report.write(f"Initial loss: {initialloss}\n")

            # training loop for first linguistic context, 10 epochs
            loss = 100000000000
            iters = 0

            # setting learning rate & multipling hidden states by b_matrix
            lr = 0.001
            # report.write(f'learning rate is {lr} \n')

            for _ in tqdm(range(500)):
              iters += 1
              loss = b_loss(optimized_hidden_vectors, distance_first_context, theta, og_hidden_vectors, A_mask)
              loss.backward(retain_graph=True)

              optimized_hidden_vectors.retain_grad()
              optimized_hidden_vectors -= lr*optimized_hidden_vectors.grad.data

              optimized_hidden_vectors.grad.data.zero_()

            new_hidden_first = torch.cat((hidden_states[0][0].unsqueeze(0), optimized_hidden_vectors, hidden_states[0][-1].unsqueeze(0)), 0).unsqueeze(0)
            oldModuleList = model.bert.encoder.layer
            newModuleList = nn.ModuleList()

            # Now iterate over all layers, only keeping only the relevant layers.
            for bert_layers in range(layer, 12):
                newModuleList.append(oldModuleList[bert_layers])

            copyOfModel = copy.deepcopy(model)
            copyOfModel.bert.encoder.layer = newModuleList
            copyOfModel.to(device)

            output_first_context = copyOfModel(inputs_embeds = new_hidden_first, return_dict = True, output_hidden_states = True)

            # print('Top 10 words')
            logits_first_context = output_first_context.logits
            softmax_first_context = F.softmax(logits_first_context, dim = -1)
            mask_word_first_context = softmax_first_context[0, mask_index, :]

            # print("BUZZZ THIS SENTENCE IS DONE")

            #here, the score metric u r creating will go here, using the all_predictions dictionary and the 5 words for each
            #context i've written

            #made a dictionary of all the context 1 words and their probabilitiess
            all_predictions = {}
            values_second_dimension = mask_word_first_context[0, :]
            num_predictions = values_second_dimension.shape[0]

            all_words = torch.topk(mask_word_first_context, num_predictions, dim = 1)[1][0]
            all_probabilites = torch.topk(mask_word_first_context, num_predictions, dim = 1)[0][0]
            counter = 0
            for token in all_words:
                word = tokenizer.decode([token])
                all_predictions[word] = all_probabilites[counter].item()
                counter += 1

            allowed_words = []
            disallowed_words = []
            if i == 0:  #first context
                allowed_words = valid_words[sent_index] #these words are for the first context
                disallowed_words = invalid_words[sent_index]
            else:
                allowed_words = invalid_words[sent_index] #these words are for the second context
                disallowed_words = valid_words[sent_index]

            minVal = min(all_predictions.values())
            for word_index in range(len(allowed_words)):
                model_allow = 0
                model_disallow = 0
                orig_allow = 0
                orig_disallow = 0

                if allowed_words[word_index] in all_predictions:
                    model_allow = all_predictions[allowed_words[word_index]]
                if allowed_words[word_index] in orig_predictions:
                    orig_allow = orig_predictions[allowed_words[word_index]]

                if disallowed_words[word_index] in all_predictions:
                    model_disallow = all_predictions[disallowed_words[word_index]]
                if disallowed_words[word_index] in orig_predictions:
                    orig_disallow = orig_predictions[disallowed_words[word_index]]

                if model_allow != 0 or orig_allow != 0:
                    increase_score += (math.log(model_allow) - math.log(orig_allow))
                    #write the original and model probabilities for the allowed words
                    report.write(f'orig_allow is {orig_allow} and model_allow is {model_allow}.\n')
                if model_disallow != 0 or orig_disallow != 0:
                    decrease_score += (math.log(orig_disallow) - math.log(model_disallow))
                    report.write(f'orig_allow is {orig_disallow} and model_allow is {model_disallow}.\n')

      print(1.0*increase_score/len(sentences) + 1.0*decrease_score/len(sentences))
      report.write(f'Average increase, decrease scores is {1.0*increase_score/len(sentences) + 1.0*decrease_score/len(sentences)}.\n')

report.close()



DEVICE cuda


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.encoder.layer.11.attention.self.value.weight', 'bert.encoder.layer.11.attention.self.key.bias', 'bert.encoder.layer.11.attention.self.query.weight', 'bert.encoder.layer.11.attention.output.dense.bias', 'bert.encoder.layer.11.intermediate.dense.weight', 'bert.encoder.layer.11.attention.self.value.bias', 'bert.encoder.layer.11.output.dense.weight', 'cls.seq_relationship.weight', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'cls.seq_relationship.bias', 'bert.encoder.layer.11.output.LayerNorm.bias', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.pooler.dense.weight', 'bert.encoder.layer.11.attention.self.query.bias', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.encoder.layer.11.attention.output.dense.weight', 'bert.encoder.layer.11.attention.self.key.weight', 'bert.encoder


NEW RUN: layer 11 rank 32 theta 0.3

index 0 this sent


<ipython-input-3-75f6973533a8>:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss2 = torch.mean(torch.tensor(A_mask, dtype=torch.float64).to(hidden_vectors.device) * (diffs - dist_context)**2)
100%|██████████| 500/500 [00:07<00:00, 66.25it/s]


index 1 this sent


...

index 56 this sent
index 56 this sent
index 56 this sent


100%|██████████| 500/500 [00:07<00:00, 63.02it/s]


100%|██████████| 500/500 [00:08<00:00, 60.23it/s]




index 57 this sent
index 57 this sent
index 57 this sent


100%|██████████| 500/500 [00:08<00:00, 58.13it/s]


100%|██████████| 500/500 [00:08<00:00, 56.06it/s]




-2.220446049250313e-16
-2.220446049250313e-16
-2.220446049250313e-16


NON RANDOM PROBES

In [ ]:
import copy
import torch
from transformers import BertTokenizerFast, BertModel, BertForMaskedLM
from torch.nn import functional as F
import torch.nn as nn
import pickle
from tqdm import tqdm
import numpy as np
import math

# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')
model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
report = open("fixed_indexing_v4.txt", "w")

with open('/content/drive/MyDrive/bertstuff/john_sentences.txt', 'r') as file:
    # Read lines from the file and store them in a list
    input_sentences = file.read().splitlines()

# obtaining distance matrix
with open('/content/drive/MyDrive/bertstuff/distance_john', 'rb') as f:
    distance_matrices = pickle.load(f)

distance_dict = {}
for i in range(len(input_sentences)):
    if input_sentences[i] not in distance_dict:
        distance_dict[input_sentences[i]] = []
    distance_dict[input_sentences[i]].append(distance_matrices[i])

def apply_transformation(matrix, vector):
    transformed_vector = torch.mm(torch.transpose(matrix, 0, 1), vector)
    return transformed_vector

valid_words = []
invalid_words = []

sentences = list(input_sentences)  #58
sentences = [sentences[i] for i in range(0, len(sentences), 2)]

import json
with open("/content/drive/MyDrive/bertstuff/dataset.json") as f:
    data = json.load(f)

for sent in data:
    attachment1_words = sent["attachment1_words"]
    attachment2_words = sent["attachment2_words"]

    min_length = min(len(attachment1_words), len(attachment2_words))
    attachment1_words = attachment1_words[:min_length]
    attachment2_words = attachment2_words[:min_length]

    valid_words.append(attachment1_words)
    invalid_words.append(attachment2_words)

def b_loss(hidden_vectors, dist_context, theta, original_hidden_vectors, A_mask):
    transformed_hidden = apply_transformation(b_matrix, hidden_vectors.permute(1, 0))  # Permute for GPU compatibility
    transformed_hidden = transformed_hidden.permute(1, 0)  # Permute back
    hidden_matrix = transformed_hidden.unsqueeze(1).expand(transformed_hidden.size(0),
                                                            transformed_hidden.size(0),
                                                            transformed_hidden.size(1))

    hidden_square_scaled = (torch.linalg.norm(dist_context)/torch.linalg.norm(hidden_matrix))*hidden_matrix

    diffs = torch.linalg.norm(hidden_square_scaled.permute(1, 0, 2) - hidden_square_scaled, dim=2)**2
    diffs.requires_grad_(True)
    loss2 = torch.mean(torch.tensor(A_mask, dtype=torch.float64).to(hidden_vectors.device) * (diffs - dist_context)**2)
    loss1 = torch.norm((hidden_vectors - original_hidden_vectors)**2, 2)

    loss = theta * loss1 + (1 - theta) * loss2
    return loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE " + str(device))


for layer in range(11,12):
  report.write(f'LAYER is {layer} \n')

  submodel = BertForMaskedLM.from_pretrained('bert-base-cased', output_hidden_states=True, return_dict = True, num_hidden_layers = 11)
  submodel.to(device)

  for rank in [64]:
    for theta in [0.3, 0.6, 1]:
      report.write(f'\nNEW RUN: layer {layer} rank {rank} theta {theta}\n')
      print(f'\nNEW RUN: layer {layer} rank {rank} theta {theta}\n')
      increase_score = 0
      decrease_score = 0

      for sent_index in range(len(sentences)):
        sent = sentences[sent_index]
        print(sent)
        print("index " + str(sent_index) + " " + "this sent")
        input = tokenizer(sent, return_tensors="pt").to(device)
        orig_output = model(**input)
        mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
        orig_logits = orig_output.logits
        orig_softmax = F.softmax(orig_logits, dim = -1)
        orig_mask_word = orig_softmax[0, mask_index, :]


        #made a dictionary of all the original BERT model words and predictions
        orig_predictions = {}

        orig_values_second_dimension = orig_mask_word[0, :]
        orig_num_predictions = orig_mask_word[0, :].shape[0]

        orig_all_words = torch.topk(orig_mask_word, orig_num_predictions, dim = 1)[1][0]
        orig_all_probabilites = torch.topk(orig_mask_word, orig_num_predictions, dim = 1)[0][0]
        orig_counter = 0
        for token in orig_all_words:
            word = tokenizer.decode([token])
            orig_predictions[word] = orig_all_probabilites[orig_counter].item()
            orig_counter += 1

        for i in range(2):
            report.write(sent + "\n")
            input = tokenizer(sent, return_tensors="pt").to(device)
            output = submodel(**input)
            mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)

            A_mask = np.not_equal(distance_dict[sent][0], distance_dict[sent][1]).astype(int)
            A_mask = torch.tensor(A_mask, device=device)

            b_matrix_path = f"/content/drive/MyDrive/bertstuff/probes/r{str(rank)}/predictor.params"
            import_b = torch.load(b_matrix_path, map_location=device)
            b_matrix = import_b['proj'].to(device)

            hidden_states = output.hidden_states[-1].to(device)
            optimized_hidden_vectors = hidden_states[0][1:-1].clone()
            og_hidden_vectors = optimized_hidden_vectors.clone()
            optimized_hidden_vectors = optimized_hidden_vectors.requires_grad_(True)
            optimized_hidden_vectors.retain_grad()

            distance_first_context = torch.tensor(distance_dict[sent][i], dtype=torch.float32, device=device)
            distance_first_context = distance_first_context**2
            distance_first_context.requires_grad_(True)

            initial_loss = b_loss(optimized_hidden_vectors, distance_first_context, theta, og_hidden_vectors, A_mask)

            # report.write(f"Initial loss: {initialloss}\n")

            # training loop for first linguistic context, 10 epochs
            loss = 100000000000
            iters = 0

            # setting learning rate & multipling hidden states by b_matrix
            lr = 0.001
            # report.write(f'learning rate is {lr} \n')

            for _ in tqdm(range(500)):
              iters += 1
              loss = b_loss(optimized_hidden_vectors, distance_first_context, theta, og_hidden_vectors, A_mask)
              loss.backward(retain_graph=True)

              optimized_hidden_vectors.retain_grad()
              optimized_hidden_vectors -= lr*optimized_hidden_vectors.grad.data

              optimized_hidden_vectors.grad.data.zero_()

            new_hidden_first = torch.cat((hidden_states[0][0].unsqueeze(0), optimized_hidden_vectors, hidden_states[0][-1].unsqueeze(0)), 0).unsqueeze(0)
            oldModuleList = model.bert.encoder.layer
            newModuleList = nn.ModuleList()

            # Now iterate over all layers, only keeping only the relevant layers.
            for bert_layers in range(layer, 12):
                newModuleList.append(oldModuleList[bert_layers])

            copyOfModel = copy.deepcopy(model)
            copyOfModel.bert.encoder.layer = newModuleList
            copyOfModel.to(device)

            output_first_context = copyOfModel(inputs_embeds = new_hidden_first, return_dict = True, output_hidden_states = True)

            # print('Top 10 words')
            logits_first_context = output_first_context.logits
            softmax_first_context = F.softmax(logits_first_context, dim = -1)
            mask_word_first_context = softmax_first_context[0, mask_index, :]

            # print("BUZZZ THIS SENTENCE IS DONE")

            #here, the score metric u r creating will go here, using the all_predictions dictionary and the 5 words for each
            #context i've written

            #made a dictionary of all the context 1 words and their probabilitiess
            all_predictions = {}
            values_second_dimension = mask_word_first_context[0, :]
            num_predictions = values_second_dimension.shape[0]

            all_words = torch.topk(mask_word_first_context, num_predictions, dim = 1)[1][0]
            all_probabilites = torch.topk(mask_word_first_context, num_predictions, dim = 1)[0][0]
            counter = 0
            for token in all_words:
                word = tokenizer.decode([token])
                all_predictions[word] = all_probabilites[counter].item()
                counter += 1

            allowed_words = []
            disallowed_words = []
            if i == 0:  #first context
                allowed_words = valid_words[sent_index] #these words are for the first context
                disallowed_words = invalid_words[sent_index]
            else:
                allowed_words = invalid_words[sent_index] #these words are for the second context
                disallowed_words = valid_words[sent_index]

            print(allowed_words)

            minVal = min(all_predictions.values())
            for word_index in range(len(allowed_words)):
                model_allow = 0
                model_disallow = 0
                orig_allow = 0
                orig_disallow = 0

                if allowed_words[word_index] in all_predictions:
                    model_allow = all_predictions[allowed_words[word_index]]
                if allowed_words[word_index] in orig_predictions:
                    orig_allow = orig_predictions[allowed_words[word_index]]

                if disallowed_words[word_index] in all_predictions:
                    model_disallow = all_predictions[disallowed_words[word_index]]
                if disallowed_words[word_index] in orig_predictions:
                    orig_disallow = orig_predictions[disallowed_words[word_index]]

                if model_allow != 0 or orig_allow != 0:
                    increase_score += (math.log(model_allow) - math.log(orig_allow))
                    #write the original and model probabilities for the allowed words
                    report.write(f'orig_allow is {orig_allow} and model_allow is {model_allow}.\n')
                if model_disallow != 0 or orig_disallow != 0:
                    decrease_score += (math.log(orig_disallow) - math.log(model_disallow))
                    report.write(f'orig_allow is {orig_disallow} and model_allow is {model_disallow}.\n')

      print(1.0*increase_score/len(sentences) + 1.0*decrease_score/len(sentences))
      report.write(f'Average increase, decrease scores is {1.0*increase_score/len(sentences) + 1.0*decrease_score/len(sentences)}.\n')

report.close()

DEVICE cuda


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.encoder.layer.11.attention.self.key.bias', 'bert.encoder.layer.11.attention.self.key.weight', 'bert.pooler.dense.weight', 'bert.encoder.layer.11.attention.self.value.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.11.intermediate.dense.weight', 'bert.encoder.layer.11.output.dense.weight', 'cls.seq_relationship.bias', 'bert.encoder.layer.11.attention.self.value.weight', 'bert.encoder.layer.11.attention.self.query.bias', 'cls.seq_relationship.weight', 'bert.encoder.layer.11.intermediate.dense.bias', 'bert.encoder.layer.11.attention.output.dense.bias', 'bert.encoder.layer.11.attention.self.query.weight', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.pooler.dense.bias', 'bert.encoder.layer.11.attention.output.dense.weight', 'bert.encod


NEW RUN: layer 11 rank 64 theta 0.3

I bought the laptop with the [MASK].
index 0 this sent


<ipython-input-4-5e2c3d2ff210>:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss2 = torch.mean(torch.tensor(A_mask, dtype=torch.float64).to(hidden_vectors.device) * (diffs - dist_context)**2)
100%|██████████| 500/500 [00:07<00:00, 65.68it/s]


['money', 'card', 'check', 'credit']


...

['footprints', 'cigarette', 'scarf']
The chef decorated the dish with the [MASK].
index 57 this sent


100%|██████████| 500/500 [00:06<00:00, 73.13it/s]


['parsley', 'lemon', 'pepper']


100%|██████████| 500/500 [00:07<00:00, 64.26it/s]


['soup', 'steak', 'pasta']
-2.220446049250313e-16


In [ ]:
!python /content/drive/MyDrive/bertstuff/direct_h_update_random_probe.py

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.encoder.layer.11.attention.self.value.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.encoder.layer.11.attention.output.dense.weight', 'bert.encoder.layer.11.output.LayerNorm.bias',

In [ ]:
!cp random_test_run_john_part_three.txt /content/drive/MyDrive/bertstuff